In [1]:
import flask
from flask import request, jsonify
from random import randint
from model_predictor import predict_maliciousness_lexical, predict_maliciousness_content
from time import time
import threading
import time
import feature_generator
from whitelist_checker import is_dom_top
import pandas as pd                     # For data transformation
import numpy as numpy                   # For scientific calculations
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, ConfusionMatrixDisplay
from xgboost import XGBClassifier, DMatrix, train
from sklearn.pipeline import Pipeline
from datetime import datetime
import joblib
import os
import optuna
from sklearn.metrics import mean_squared_error # or any other metric
from sklearn.model_selection import train_test_split
import machine_learning
import database_operations
import feature_engineering

c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
valid = pd.read_csv("../02_feature-engineering/final-datasets/validation_no_features.csv")
valid.head()

y_valid = valid['Type']
X_urls = valid.drop(columns=['Type'])

In [3]:
url_list = X_urls['URL'].tolist()
print(url_list)

['https://www.google.com', 'https://www.youtube.com', 'https://www.facebook.com', 'https://www.amazon.com', 'https://www.wikipedia.org', 'https://www.twitter.com', 'https://www.instagram.com', 'https://www.linkedin.com', 'https://www.reddit.com', 'https://www.netflix.com', 'https://www.microsoft.com', 'https://www.apple.com', 'https://www.pinterest.com', 'https://www.tumblr.com', 'https://www.ebay.com', 'https://www.bing.com', 'https://www.cnn.com', 'https://www.bbc.com', 'https://www.yahoo.com', 'https://www.nytimes.com', 'https://www.weather.com', 'https://www.quora.com', 'https://www.stackoverflow.com', 'https://www.github.com', 'https://www.medium.com', 'https://www.spotify.com', 'https://www.dropbox.com', 'https://www.paypal.com', 'https://www.salesforce.com', 'https://www.airbnb.com', 'https://www.adobe.com', 'https://www.udemy.com', 'https://www.coursera.org', 'https://www.khanacademy.org', 'https://www.walmart.com', 'https://www.homedepot.com', 'https://www.target.com', 'https:

In [4]:
y_valid.head()

0    0
1    0
2    0
3    0
4    0
Name: Type, dtype: int64

In [5]:
important_features_wrapper_lexical = ['url_length',
 'url_domain_entropy',
 'url_is_digits_in_domain',
 'url_number_of_parameters',
 'url_number_of_digits',
 'url_string_entropy',
 'url_path_length',
 'url_host_length',
 'url_domain_len',
 'url_num_subdomain',
 'url_number_of_fragments',
 'url_is_encoded',
 'url_number_of_letters',
 'url_num_periods',
 'url_num_of_hyphens',
 'url_num_underscore',
 'url_num_forward_slash',
 'url_num_semicolon',
 'url_num_mod_sign',
 'has_login_in_string',
 'has_signin_in_string',
 'has_logon_in_string',
 'has_loginasp_in_string',
 'has_exe_in_string',
 'has_viewerphp_in_string',
 'has_getImageasp_in_string',
 'has_paypal_in_string',
 'has_dbsysphp_in_string',
 'has_shopping_in_string',
 'has_php_in_string',
 'has_bin_in_string',
 'has_personal_in_string'
 ]

important_features_wrapper = ['blank_lines_count', 
                        'word_count', 
                        'js_count', 
                        'js_find_count', 
                        'js_link_count', 
                        'js_winopen_count', 
                        'title_tag_presence', 
                        'meta_tag_count', 
                        'anchor_tag_count', 
                        'applet_tag_count', 
                        'input_tag_count', 
                        'has_free_in_html', 
                        'has_access_in_html', 
                        'url_length', 
                        'url_ip_in_domain', 
                        'url_domain_entropy', 
                        'url_is_digits_in_domain', 
                        'url_number_of_digits', 
                        'url_string_entropy', 
                        'url_path_length', 
                        'url_host_length', 
                        'url_number_of_subdirectories', 
                        'url_num_subdomain', 
                        'url_has_port', 
                        'url_number_of_fragments', 
                        'url_num_periods', 
                        'url_num_equal', 
                        'url_num_open_parenthesis', 
                        'url_num_close_parenthesis', 
                        'url_num_ampersand', 
                        'url_num_at', 
                        'has_account_in_string', 
                        'has_webscr_in_string', 
                        'has_ebayisapi_in_string', 
                        'has_signin_in_string', 
                        'has_banking_in_string', 
                        'has_confirm_in_string', 
                        'has_logon_in_string', 
                        'has_signon_in_string', 
                        'has_loginasp_in_string', 
                        'has_loginphp_in_string', 
                        'has_exe_in_string', 
                        'has_zip_in_string', 
                        'has_rar_in_string', 
                        'has_jpg_in_string', 
                        'has_gif_in_string', 
                        'has_viewerphp_in_string', 
                        'has_getImageasp_in_string', 
                        'has_plugins_in_string', 
                        'has_paypal_in_string', 
                        'has_dbsysphp_in_string', 
                        'has_configbin_in_string', 
                        'has_downloadphp_in_string', 
                        'has_payment_in_string', 
                        'has_files_in_string', 
                        'has_shopping_in_string', 
                        'has_mailphp_in_string', 
                        'has_jar_in_string', 
                        'has_swf_in_string', 
                        'has_cgi_in_string', 
                        'has_php_in_string', 
                        'has_abuse_in_string', 
                        'has_bin_in_string', 
                        'has_update_in_string', 
                        'has_verification_in_string']

In [28]:
pipeline = joblib.load("model/xgb_wrapper_65_lexical-content.sav")

In [35]:
y_pred = []

for i in range(1):
    try:
        
        # inp_url = url.replace("https://","",1)
        # inp_url = url.replace("http://","",1)
        
        url_features_pandas = feature_generator.content_generator(important_features_wrapper, "www.facebook.com/")
        url_features = DMatrix(url_features_pandas)

        prediction = pipeline.predict(url_features)

        y_pred.append(prediction)
    except:
        y_pred.append(1)

c:\Users\sison\OneDrive - University of the Philippines\04_Senior-Year_23-24\02_Acad-Files\01_EEE\01_EEE-196_199\00_Actual-Repo\SecURL\first_flask\feature_generation_content_function_htmlin.py:66: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html, features = 'html.parser')
c:\Users\sison\OneDrive - University of the Philippines\04_Senior-Year_23-24\02_Acad-Files\01_EEE\01_EEE-196_199\00_Actual-Repo\SecURL\first_flask\feature_generation_content_function_htmlin.py:149: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html, features = 'html.parser')
c:\Users\sison\OneDrive - University of the Philippines\04_Senior-Year_23-24\02_Acad-Files\01_EEE\01_EEE-196_199\00_Actual-Repo\SecURL\first_flask\feature_generation_content_functio

In [36]:
print(y_pred)

[array([0.], dtype=float32)]


In [12]:
print(classification_report(y_valid, y_pred))

ValueError: Classification metrics can't handle a mix of binary and unknown targets